In [1]:
from psycopg2 import OperationalError
import pandas as pd
import psycopg2
import os

## Functions for interaction with PostgreSQL

In [2]:
def create_connection(db_name, db_user, db_password, db_host, db_port):
    
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
        
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    
    return connection

In [3]:
def create_database(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Creating table query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [4]:
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [5]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except OperationalError as e:
        print(f"The error '{e}' occurred")

## Connect to PostgreSQL

In [6]:
db_name = "eppmjgdg" 
db_user = "eppmjgdg"
db_password = "TPGnoBvXCNr3stnQviJ4R4050Fd2xUQ_"
db_host = "hattie.db.elephantsql.com"
db_port = "5432"

In [7]:
connection = create_connection(
    db_name = db_name, 
    db_user = db_user, 
    db_password = db_password, 
    db_host = db_host, 
    db_port = db_port
)

Connection to PostgreSQL DB successful


## Functions for reading and cleaning tables

In [8]:
def make_clear_data(table):
    
    for i in range(len(table)):
        for column in list(table.columns)[2:]:
            try:
                float(table.loc[i, column])
            except:
                table = table.drop([i])
                break
                
    for column in list(table.columns)[2:]:
        table[column] = table[column].astype('float')
    
    table = table.reset_index()
    table = table.drop(columns=['index'])
    
    return table

In [9]:
def make_types_dict(table):
    
    types_dict = {}
    types_dict['id'] = 'SERIAL'

    for column in list(table.columns):

        #print(f'{column}: {table[column].dtype}')

        if table[column].dtype == 'float64':
            column = column.replace("&", " and ").replace(")", "").replace("(", "").replace(".", "").replace("/", "_").replace("-", "").replace(" ", "_")
            types_dict[column] = 'REAL'
        elif table[column].dtype == 'object':
            column = column.replace("&", " and ").replace(")", "").replace("(", "").replace(".", "").replace("/", "_").replace("-", "").replace(" ", "_")
            types_dict[column] = 'TEXT'
            
    return types_dict

In [10]:
def make_table_creating_query(types_dict, table_name):
    create_table = f'CREATE TABLE {table_name}('

    for i, column in enumerate(list(types_dict.keys())):
        if i != len(list(types_dict.keys())) - 1:
            if i == 0:
                create_table += f'\n  {column} {types_dict[column]} PRIMARY KEY,'
            else:
                create_table += f'\n  {column} {types_dict[column]},'
        else:
            create_table += f'\n  {column} {types_dict[column]}'

    create_table += '\n)'
    
    return create_table

In [11]:
def make_rows(table):
    rows = []

    for i in range(len(table)):
        rows.append(tuple(table.loc[i]))
        
    return rows

In [12]:
def make_insert_query(connection, rows, table_name):
    
    rows_records = ", ".join(["%s"] * len(rows))

    insert_query = (
        f"INSERT INTO {table_name} ({', '.join(list(types_dict.keys())[1:])}) VALUES {rows_records}"
    )

    connection.autocommit = True
    cursor = connection.cursor()
    cursor.execute(insert_query, rows)
    
    print("Query executed successfully")

## Make tables on PostgreSQL

In [13]:
path = './data/main tables/single tables/'
files = list(os.walk(path))[0][2]
files

['quarterly_balance_sheet.xlsx',
 'quarterly_cash_flow.xlsx',
 'quarterly_income_statements.xlsx',
 'yearly_balance_sheet.xlsx',
 'yearly_cash_flow.xlsx',
 'yearly_income_statements.xlsx']

In [14]:
for file in files:
    
    table = pd.read_excel(f'{path}{file}')
    table = table.drop(columns=['Unnamed: 0'])
    
    table_name = file[:-5]
    
    table = make_clear_data(table)
    types_dict = make_types_dict(table)
    
    creating_table_query = make_table_creating_query(types_dict, table_name)
    execute_query(connection, creating_table_query)
    
    rows = make_rows(table)
    ind = len(rows) // 2
    
    make_insert_query(connection, rows[:ind], table_name)
    make_insert_query(connection, rows[ind:], table_name)
    
    print(f'{file[:-5]} - DONE!')
    print('=' * 100)

Query executed successfully
Query executed successfully
Query executed successfully
quarterly_balance_sheet - DONE!
Query executed successfully
Query executed successfully
Query executed successfully
quarterly_cash_flow - DONE!
Query executed successfully
Query executed successfully
Query executed successfully
quarterly_income_statements - DONE!
Query executed successfully
Query executed successfully
Query executed successfully
yearly_balance_sheet - DONE!
Query executed successfully
Query executed successfully
Query executed successfully
yearly_cash_flow - DONE!
Query executed successfully
Query executed successfully


KeyboardInterrupt: 